# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd


In [9]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM message', engine)
X = df.message
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [10]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [119]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(KNeighborsClassifier())),
                ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [120]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size =.3,random_state = 42)


In [121]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [122]:
from sklearn.metrics import classification_report

def get_scores(y_pred,y_test):
    result = []
    for i in range(y_test.shape[1]): 
        test_value = y_test.iloc[:, i]
        pred_value = [a[i] for a in y_pred]
        result.append(list(classification_report(test_value, pred_value,output_dict = True)['0'].values())[:3])
    return pd.DataFrame(result,columns=['precision','recall','f1_score']).mean()

    
    

In [124]:
KNeighborsClassifier_score = get_scores(y_pred,y_test)
KNeighborsClassifier_score

precision    0.931183
recall       0.973001
f1_score     0.950552
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [100]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(KNeighborsClassifier())),
                ])

pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a1cd2d290>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',
                                                        leaf_size=30,
                                                        metric='minkowski',
                                                        metric_params=None,
                                                        n_jobs=None, n_neighbors=5,
           

In [194]:
from sklearn.model_selection import GridSearchCV

parameters = { 
    'vect__max_df':[0.5,1.0],
    'clf__estimator__n_neighbors':[3,5,7],
    'clf__estimator__leaf_size':[20,30,40],    
    }

cv = GridSearchCV(pipeline, param_grid=parameters, cv = 2, n_jobs = -1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [131]:
# fit the train data
cv.fit(X_train, y_train)
# get the pred value
y_pred = cv.predict(X_test)



In [132]:
cv.cv_results_

{'mean_fit_time': array([4.54942596, 4.61293054, 3.07861483, 3.14130759, 3.50891495,
        3.49740267, 3.32925618, 3.30847251, 3.14226794, 3.09994531,
        3.33424139, 3.28952599, 3.11634195, 3.47763455, 3.52636051,
        3.34596908, 3.30294824, 3.3062259 ]),
 'std_fit_time': array([0.10222399, 0.04877734, 0.04008615, 0.04157448, 0.02964711,
        0.1558845 , 0.03394496, 0.03111446, 0.03133202, 0.02516842,
        0.03896141, 0.00840712, 0.04161489, 0.05200255, 0.09935045,
        0.07057893, 0.02718258, 0.06511796]),
 'mean_score_time': array([137.42053699, 160.08084941, 146.95691717, 166.46277857,
        156.90371966, 176.8538239 , 124.36755335, 145.87705302,
        150.89438033, 165.57653749, 142.56114793, 200.483881  ,
        159.44192851, 183.32705736, 183.17842948, 171.36719728,
        154.1842221 , 134.82823122]),
 'std_score_time': array([0.14189005, 0.04880738, 1.32731521, 1.90153146, 2.95004249,
        1.12354803, 0.50455534, 0.32984805, 5.61507058, 1.49859846,


In [135]:
cv.best_params_

{'clf__estimator__leaf_size': 20,
 'clf__estimator__n_neighbors': 5,
 'vect__max_df': 0.5}

In [138]:
# get the three scores of the best combination
get_scores(y_pred,y_test)

precision    0.929722
recall       0.978194
f1_score     0.952168
dtype: float64

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [169]:
#first, try to add a text length feature and a starting verb feature
import nltk
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def textlength(self, text):        
        return len(tokenize(text))

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.textlength)
        return pd.DataFrame(X_tagged)


class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)



In [197]:
pipeline_union = Pipeline([
    ('features', FeatureUnion([

        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize,max_df =0.5)),
            ('tfidf', TfidfTransformer())
             ])),

        ('txt_len', TextLengthExtractor()),
        ('start_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(KNeighborsClassifier(leaf_size =20)))
])



In [198]:
pipeline_union.fit(X_train, y_train)
y_pred_union = pipeline_union.predict(X_test)

In [200]:
get_scores(y_pred_union,y_test)

precision    0.934191
recall       0.957524
f1_score     0.944294
dtype: float64

In [179]:
# try randomforestclassifier
from sklearn.ensemble import RandomForestClassifier

pipeline_rf = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier())),
                ])

pipeline_rf.fit(X_train, y_train)
y_pred_rf = pipeline_rf.predict(X_test)

In [181]:
get_scores(y_pred_rf,y_test)

precision    0.944321
recall       0.973485
f1_score     0.955139
dtype: float64

In [182]:
pipeline_rf.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a1cd2d290>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                 

In [183]:
parameters = { 
    'vect__max_df':[0.5,1.0],
    'clf__estimator__min_samples_leaf':[1,10,100],
    'clf__estimator__n_estimators':[10,100,1000],    
    }

cv_rf = GridSearchCV(pipeline_rf, param_grid=parameters, cv = 2, n_jobs = -1)

In [184]:
# fit the train data
cv_rf.fit(X_train, y_train)
# get the pred value
y_pred_rf = cv_rf.predict(X_test)



/Users/xuhao3/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [185]:
cv_rf.cv_results_

{'mean_fit_time': array([  14.9142375 ,   15.11135793,  103.81421053,  105.32928455,
        1050.12543797, 1077.2287184 ,    6.2204361 ,    6.08525693,
          31.27337515,   31.13991296,  271.02321637,  271.62071645,
           5.26762342,    5.27831447,   10.80115867,   10.93074942,
          81.16829395,   82.49217212]),
 'std_fit_time': array([1.97382450e-01, 2.30489016e-01, 2.47808564e+00, 1.98966563e+00,
        2.72807801e+01, 3.34113194e+01, 1.36315107e-01, 1.67208910e-02,
        6.11735940e-01, 6.28281832e-02, 8.87270570e-01, 5.10180593e-01,
        3.50325108e-02, 4.67332602e-02, 6.94355965e-02, 4.04474735e-02,
        1.15309000e-01, 1.98697925e-01]),
 'mean_score_time': array([  5.28376389,   5.20149052,  27.16379309,  26.39425647,
        560.57951701, 534.6114856 ,   4.24459434,   4.09834063,
         13.91008496,  13.09914851, 107.33869994, 100.79286802,
          3.21042144,   3.19971061,   5.97512281,   6.18708014,
         31.39809954,  32.22653484]),
 'std_score_

In [186]:
cv_rf.best_params_

{'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__n_estimators': 1000,
 'vect__max_df': 0.5}

In [188]:
get_scores(y_pred_rf,y_test)

precision    0.945068
recall       0.973044
f1_score     0.955248
dtype: float64

In [232]:
pipeline_union2 = Pipeline([
    ('features', FeatureUnion([

        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, max_df = 0.5)),
            ('tfidf', TfidfTransformer())
             ])),

        ('txt_len', TextLengthExtractor()),
        ('start_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_leaf =1,n_estimators = 1000)))
])

In [233]:
now = time.time()
pipeline_union2.fit(X_train, y_train)
y_pred_union2 = pipeline_union2.predict(X_test)
time_cost = time.time() - now

In [235]:
get_scores(y_pred_union2,y_test)

precision    0.945194
recall       0.972778
f1_score     0.955031
dtype: float64

In [236]:
time_cost

3095.8152940273285

In [213]:
# try naive bayes classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import FunctionTransformer

pipeline_nb = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('todense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
                ('clf', MultiOutputClassifier(GaussianNB())),
                ])

pipeline_nb.fit(X_train, y_train)
y_pred_nb = pipeline_nb.predict(X_test)



In [214]:
get_scores(y_pred_nb,y_test)

precision    0.929827
recall       0.778979
f1_score     0.842159
dtype: float64

In [231]:
import time

a = time.time()
a
for i in range(1000):
    i += 1
b = time.time() - a
b

0.00014901161193847656

In [237]:
pipeline_randomfo = Pipeline([
    ('features', FeatureUnion([

        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize, max_df = 0.5)),
            ('tfidf', TfidfTransformer())
             ])),

        ('txt_len', TextLengthExtractor()),
        ('start_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_leaf =1,n_estimators = 1000)))
])

In [238]:
pipeline_randomfo.get_params()

{'memory': None,
 'steps': [('features', FeatureUnion(n_jobs=None,
                transformer_list=[('nlp_pipeline',
                                   Pipeline(memory=None,
                                            steps=[('vect',
                                                    CountVectorizer(analyzer='word',
                                                                    binary=False,
                                                                    decode_error='strict',
                                                                    dtype=<class 'numpy.int64'>,
                                                                    encoding='utf-8',
                                                                    input='content',
                                                                    lowercase=True,
                                                                    max_df=0.5,
                                                                    max_fea

In [239]:
parameters = {
        'features__nlp_pipeline__vect__max_features': (5000, 10000),
        'features__nlp_pipeline__tfidf__use_idf': (True, False),
        'clf__estimator__min_samples_split': [2, 4],
        }


In [240]:
cv_randomfo = GridSearchCV(pipeline_randomfo, param_grid=parameters, cv = 2, n_jobs = -1)

In [ ]:
now = time.time()
# fit the train data
cv_randomfo.fit(X_train, y_train)
# get the pred value
y_pred_randomfo = cv_randomfo.predict(X_test)

time_cost = time.time() - now

### 9. Export your model as a pickle file

In [ ]:
import pickle
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import sys
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def load_data(database_filepath):
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql('SELECT * FROM message', engine)
    X = df.message
    y = df.iloc[:, 4:]
    category_names = list(y.columns)
    return X, y, category_names

def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



def build_model():
    pass


def evaluate_model(model, X_test, Y_test, category_names):
    pass


def save_model(model, model_filepath):
    pass


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()